# Weather feature extraction for specific location and date

In [1]:
#Imporing packages

import pandas as pd
import sys, os

sys.path.append(os.path.abspath(os.path.join('../utils')))

In [53]:
#import modules
from eda_helpers import EDA
from weather_api_connecter import HistoricalWeatherFetcher
from data_processing import DataProcessor

In [3]:
eda = EDA('../data/processed')
eda.load_cleaned_data()

In [54]:
#instantiate Classes

preprocessed_trip_df=eda.preprocessed_trip
weather_data_loader=HistoricalWeatherFetcher()


data_processer=DataProcessor()


In [5]:
preprocessed_trip_df = preprocessed_trip_df.drop('Unnamed: 0', axis=1)

In [ ]:
preprocessed_trip_df

In [7]:
w = HistoricalWeatherFetcher()
trip_origin = (6.508813001668548, 3.377403)
trip_start_time = "2021-07-01 07:28:04"
weather_condition, temp_classification = w.get_daily_weather(trip_origin, trip_start_time)
print(f"Weather Condition: {weather_condition}")
print(f"Temperature Classification: {temp_classification}")

Weather Condition: Rainy
Temperature Classification: High


In [13]:
trip_df=preprocessed_trip_df.copy()

In [15]:
trip_df['Trip Start Time'] = pd.to_datetime(trip_df['Trip Start Time'])
print(trip_df['Trip Start Time'].dt.date.nunique())

252


In [16]:
trip_df['Trip Start Time'] = pd.to_datetime(trip_df['Trip Start Time'])


In [ ]:
trip_df

In [20]:
unique_trips = trip_df.assign(date=trip_df['Trip Start Time'].dt.date)[['date']].drop_duplicates().reset_index(drop=True)

In [21]:
unique_trips

,date
0,2021-07-01
1,2021-07-02
2,2021-07-04
3,2021-07-03
4,2021-07-07
...,...
247,2022-02-14
248,2022-03-11
249,2022-01-20
250,2022-01-02


### make weather api call for unique days and save the rain and temperature data

In [22]:
# unique_trips = pd.DataFrame({'date': ['2021-07-01 07:28:04']})
unique_trips['Rain_Condition'] = None
unique_trips['Temperature'] = None

w = HistoricalWeatherFetcher()

trip_origin_str = '(6.508813001668548, 3.37740316890347)'
trip_origin = eval(trip_origin_str)  # convert string to tuple

for index, row in unique_trips.iterrows():
    trip_start_time = row['date'].strftime('%Y-%m-%d %H:%M:%S')
    weather_condition, temp_classification = w.get_daily_weather(trip_origin, trip_start_time)
    unique_trips.loc[index, 'Rain_Condition'] = weather_condition
    unique_trips.loc[index, 'Temperature'] = temp_classification

print(unique_trips)

           date Rain_Condition Temperature
0    2021-07-01          Rainy        High
1    2021-07-02          Rainy        High
2    2021-07-04          Rainy    Moderate
3    2021-07-03          Rainy    Moderate
4    2021-07-07          Rainy    Moderate
..          ...            ...         ...
247  2022-02-14  Partly Cloudy        High
248  2022-03-11         Cloudy        High
249  2022-01-20  Partly Cloudy        High
250  2022-01-02  Partly Cloudy        High
251  2022-03-17         Cloudy        High

[252 rows x 3 columns]


In [23]:
unique_trips

,date,Rain_Condition,Temperature
0,2021-07-01,Rainy,High
1,2021-07-02,Rainy,High
2,2021-07-04,Rainy,Moderate
3,2021-07-03,Rainy,Moderate
4,2021-07-07,Rainy,Moderate
...,...,...,...
247,2022-02-14,Partly Cloudy,High
248,2022-03-11,Cloudy,High
249,2022-01-20,Partly Cloudy,High
250,2022-01-02,Partly Cloudy,High


In [27]:
import numpy as np

unique_trips['Rain_Condition'] = np.where(unique_trips['Rain_Condition'] == 'Rainy', 'Rainy', 'Not Rainy')

In [29]:
print(unique_trips['Rain_Condition'].value_counts())
print(unique_trips['Temperature'].value_counts())

Rain_Condition
Rainy        130
Not Rainy    122
Name: count, dtype: int64
Temperature
High        193
Moderate     59
Name: count, dtype: int64


In [30]:
unique_trips.to_csv("../data/processed/unique_trips_weather.csv")

In [31]:
trip_df

,order_id,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,is_weekend,is_holiday,distance,speed
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,0,0,16.123451,624.133590
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,0,0,1.884305,3.845521
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,0,0,13.822393,20.056676
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,0,0,1.511034,6.674507
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952
...,...,...,...,...,...,...,...,...,...
528415,1637696,"6.448218499999999,3.4772075","6.437787399999999,3.481670199999999",2021-12-30 20:35:06,2021-12-30 21:02:59,0,0,1.254752,2.700005
528416,1637702,"6.442320899999999,3.4736868","6.436589333407897,3.5559738188407835",2021-12-30 20:48:13,2021-12-30 21:43:49,0,0,9.124781,9.846886
528417,1637704,"6.4281982,3.492248","6.448088500000001,3.4775747",2021-12-30 20:51:45,2021-12-30 21:41:32,0,0,2.733704,3.294722
528418,1637705,"6.5869296,3.3632966","6.637906899999999,3.3339515",2021-12-30 20:48:50,2021-12-30 21:08:28,0,0,6.504796,19.878833


In [35]:
unique_trips = unique_trips.rename(columns={'Trip Start Time': 'date'})
unique_trips

,date,Rain_Condition,Temperature
0,2021-07-01,Rainy,High
1,2021-07-02,Rainy,High
2,2021-07-04,Rainy,Moderate
3,2021-07-03,Rainy,Moderate
4,2021-07-07,Rainy,Moderate
...,...,...,...
247,2022-02-14,Not Rainy,High
248,2022-03-11,Not Rainy,High
249,2022-01-20,Not Rainy,High
250,2022-01-02,Not Rainy,High


In [36]:
trip_df['date'] = trip_df['Trip Start Time'].dt.date
trip_df

,order_id,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,is_weekend,is_holiday,distance,speed,date
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,0,0,16.123451,624.133590,2021-07-01
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,0,0,1.884305,3.845521,2021-07-01
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,0,0,13.822393,20.056676,2021-07-01
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,0,0,1.511034,6.674507,2021-07-01
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01
...,...,...,...,...,...,...,...,...,...,...
528415,1637696,"6.448218499999999,3.4772075","6.437787399999999,3.481670199999999",2021-12-30 20:35:06,2021-12-30 21:02:59,0,0,1.254752,2.700005,2021-12-30
528416,1637702,"6.442320899999999,3.4736868","6.436589333407897,3.5559738188407835",2021-12-30 20:48:13,2021-12-30 21:43:49,0,0,9.124781,9.846886,2021-12-30
528417,1637704,"6.4281982,3.492248","6.448088500000001,3.4775747",2021-12-30 20:51:45,2021-12-30 21:41:32,0,0,2.733704,3.294722,2021-12-30
528418,1637705,"6.5869296,3.3632966","6.637906899999999,3.3339515",2021-12-30 20:48:50,2021-12-30 21:08:28,0,0,6.504796,19.878833,2021-12-30


### Merge the weather data with other columns

In [ ]:
unique_trips['Trip Start Time'] = pd.to_datetime(unique_trips['Trip Start Time'])
trip_df['Trip Start Time'] = pd.to_datetime(trip_df['Trip Start Time'], format='%Y-%m-%d %H:%M:%S')
merged_data = pd.merge(trip_df, unique_trips, on='Trip Start Time')

In [44]:
trip_merged_df = pd.merge(trip_df, unique_trips, on='date', how='inner')

In [45]:
trip_merged_df

,order_id,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,is_weekend,is_holiday,distance,speed,date,Rain_Condition,Temperature
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,0,0,16.123451,624.133590,2021-07-01,Rainy,High
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,0,0,1.884305,3.845521,2021-07-01,Rainy,High
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,0,0,13.822393,20.056676,2021-07-01,Rainy,High
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,0,0,1.511034,6.674507,2021-07-01,Rainy,High
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01,Rainy,High
...,...,...,...,...,...,...,...,...,...,...,...,...
528415,1637696,"6.448218499999999,3.4772075","6.437787399999999,3.481670199999999",2021-12-30 20:35:06,2021-12-30 21:02:59,0,0,1.254752,2.700005,2021-12-30,Not Rainy,High
528416,1637702,"6.442320899999999,3.4736868","6.436589333407897,3.5559738188407835",2021-12-30 20:48:13,2021-12-30 21:43:49,0,0,9.124781,9.846886,2021-12-30,Not Rainy,High
528417,1637704,"6.4281982,3.492248","6.448088500000001,3.4775747",2021-12-30 20:51:45,2021-12-30 21:41:32,0,0,2.733704,3.294722,2021-12-30,Not Rainy,High
528418,1637705,"6.5869296,3.3632966","6.637906899999999,3.3339515",2021-12-30 20:48:50,2021-12-30 21:08:28,0,0,6.504796,19.878833,2021-12-30,Not Rainy,High


In [46]:
trip_merged_df['Temperature'].value_counts()

Temperature
High        371559
Moderate    156861
Name: count, dtype: int64

## Merge the trip data with the features to the request data

In [43]:
eda.load_cleaned_data()

cleaned_requests_df = eda.cleaned_requests
cleaned_requests_df

,id,order_id,driver_id,driver_action,lat,lng
0,1,392001,243828,accepted,6.602207,3.270465
1,2,392001,243588,rejected,6.592097,3.287445
2,3,392001,243830,rejected,6.596133,3.281784
3,4,392001,243539,rejected,6.596142,3.280526
4,5,392001,171653,rejected,6.609232,3.288800
...,...,...,...,...,...,...
1557735,1557736,517948,243774,rejected,6.469036,3.566877
1557736,1557737,517948,245447,rejected,6.425431,3.527358
1557737,1557738,517948,239866,rejected,6.440013,3.525378
1557738,1557739,517948,243774,rejected,6.469036,3.566877


In [47]:
#extract drivers location in (lat,lng) format to calculate the distance

cleaned_requests_df['Drivers Location'] = cleaned_requests_df.apply(lambda row: f"{row['lat']}, {row['lng']}", axis=1)

In [48]:
cleaned_requests_df

,id,order_id,driver_id,driver_action,lat,lng,Drivers Location
0,1,392001,243828,accepted,6.602207,3.270465,"6.6022066, 3.2704649"
1,2,392001,243588,rejected,6.592097,3.287445,"6.5920972, 3.2874447"
2,3,392001,243830,rejected,6.596133,3.281784,"6.5961334, 3.2817841"
3,4,392001,243539,rejected,6.596142,3.280526,"6.5961416, 3.2805263"
4,5,392001,171653,rejected,6.609232,3.288800,"6.6092317, 3.2887999"
...,...,...,...,...,...,...,...
1557735,1557736,517948,243774,rejected,6.469036,3.566877,"6.469036, 3.5668767"
1557736,1557737,517948,245447,rejected,6.425431,3.527358,"6.4254311, 3.5273581"
1557737,1557738,517948,239866,rejected,6.440013,3.525378,"6.4400128, 3.5253785"
1557738,1557739,517948,243774,rejected,6.469036,3.566877,"6.469036, 3.5668767"


### Merge request and trip preprocessed data based on order_id 

In [49]:
trip_requests_merged_df = pd.merge(cleaned_requests_df, trip_merged_df, on='order_id', how='inner')

In [50]:
trip_requests_merged_df

,id,order_id,driver_id,driver_action,lat,lng,Drivers Location,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,is_weekend,is_holiday,distance,speed,date,Rain_Condition,Temperature
0,1,392001,243828,accepted,6.602207,3.270465,"6.6022066, 3.2704649","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01,Rainy,High
1,2,392001,243588,rejected,6.592097,3.287445,"6.5920972, 3.2874447","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01,Rainy,High
2,3,392001,243830,rejected,6.596133,3.281784,"6.5961334, 3.2817841","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01,Rainy,High
3,4,392001,243539,rejected,6.596142,3.280526,"6.5961416, 3.2805263","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01,Rainy,High
4,5,392001,171653,rejected,6.609232,3.288800,"6.6092317, 3.2887999","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01,Rainy,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1551516,1557736,517948,243774,rejected,6.469036,3.566877,"6.469036, 3.5668767","6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,0,0,8.112927,4.735939,2021-09-10,Rainy,High
1551517,1557737,517948,245447,rejected,6.425431,3.527358,"6.4254311, 3.5273581","6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,0,0,8.112927,4.735939,2021-09-10,Rainy,High
1551518,1557738,517948,239866,rejected,6.440013,3.525378,"6.4400128, 3.5253785","6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,0,0,8.112927,4.735939,2021-09-10,Rainy,High
1551519,1557739,517948,243774,rejected,6.469036,3.566877,"6.469036, 3.5668767","6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,0,0,8.112927,4.735939,2021-09-10,Rainy,High


In [51]:
print(trip_requests_merged_df.isnull().sum())

id                  0
order_id            0
driver_id           0
driver_action       0
lat                 0
lng                 0
Drivers Location    0
Trip Origin         0
Trip Destination    0
Trip Start Time     0
Trip End Time       0
is_weekend          0
is_holiday          0
distance            0
speed               0
date                0
Rain_Condition      0
Temperature         0
dtype: int64


## Calculate distance from driver to Trip

In [55]:
start_coordinates = trip_requests_merged_df['Trip Origin'].tolist()
end_coordinates = trip_requests_merged_df['Drivers Location'].tolist()

In [56]:
trip_requests_merged_df['Driver distance'] = data_processer.calculate_distances(start_coordinates, end_coordinates)

In [57]:
trip_requests_merged_df

,id,order_id,driver_id,driver_action,lat,lng,Drivers Location,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,is_weekend,is_holiday,distance,speed,date,Rain_Condition,Temperature,Driver distance
0,1,392001,243828,accepted,6.602207,3.270465,"6.6022066, 3.2704649","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01,Rainy,High,0.694264
1,2,392001,243588,rejected,6.592097,3.287445,"6.5920972, 3.2874447","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01,Rainy,High,1.551694
2,3,392001,243830,rejected,6.596133,3.281784,"6.5961334, 3.2817841","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01,Rainy,High,0.786777
3,4,392001,243539,rejected,6.596142,3.280526,"6.5961416, 3.2805263","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01,Rainy,High,0.692054
4,5,392001,171653,rejected,6.609232,3.288800,"6.6092317, 3.2887999","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01,Rainy,High,1.621848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1551516,1557736,517948,243774,rejected,6.469036,3.566877,"6.469036, 3.5668767","6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,0,0,8.112927,4.735939,2021-09-10,Rainy,High,3.263377
1551517,1557737,517948,245447,rejected,6.425431,3.527358,"6.4254311, 3.5273581","6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,0,0,8.112927,4.735939,2021-09-10,Rainy,High,3.399641
1551518,1557738,517948,239866,rejected,6.440013,3.525378,"6.4400128, 3.5253785","6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,0,0,8.112927,4.735939,2021-09-10,Rainy,High,3.002545
1551519,1557739,517948,243774,rejected,6.469036,3.566877,"6.469036, 3.5668767","6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,0,0,8.112927,4.735939,2021-09-10,Rainy,High,3.263377


### Save the merged data

In [58]:
trip_requests_merged_df.to_csv("../data/processed/trip_requests_merged.csv")